## Python and R

This setup allows you to use *Python* and *R* in the same notebook.

To set up a similar notebook, see quickstart instructions here:

https://github.com/dmil/jupyter-quickstart

Some thoughts on why I like this setup and how I use it at the [end](notebook.ipynb#Thoughts) of  this notebook.

In [ ]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

# always show all columns
pd.set_option('display.max_columns', None)

This is a Python notebook, but below is an R cell. The `%%R` at the top of the cell indicates that the code in this cell will be R code.

In [ ]:
%%R

# My commonly used R imports

require('tidyverse')

## Read Election Data

Load election results

In [ ]:
election_results = pd.read_csv('data/election_results.csv', dtype={'fips': str})

Filter to presidential only

In [ ]:
# filter to where type is presidential only
election_results = election_results[election_results['type'] == 'presidential']

Make a column for 2020 results 

In [ ]:
last_election = election_results[election_results['year'] == 2020]

election_results = election_results.merge(
    last_election[['fips', 'democratic_pct', 'republican_pct']],
    on='fips',
    how='left',
    suffixes=('', '_2020')
)

**Filter to 2024 only**

So we are left with 2024 presidential election results with a column containing 2020 results

In [ ]:

# filter to 2024 only
election_results = election_results[election_results['year'] == 2024]
election_results

## Get Census Data

In [ ]:
%%R 

require(tidycensus)
options(tigris_use_cache = TRUE)
# Read API key from .env
Sys.setenv(CENSUS_API_KEY = Sys.getenv("CENSUS_API_KEY"))
# Read census data
census_data = get_acs(geography = "county", 
    # get population, 
    variables = c(
        total_pop = "B01003_001",
        median_income = "B19013_001",
        median_age = "B01002_001",
        white = "B02001_002",
        black = "B02001_003",
        native = "B02001_004",
        asian = "B02001_005",
        hispanic = "B03002_012",
        below_poverty = "B17001_002"),
    year = 2022,
    geometry = T)


pivot and calculate percents

In [ ]:
%%R

census_data <- census_data %>% 
    pivot_wider(
        id_cols=c(GEOID, NAME, geometry),
        names_from = variable, 
        values_from = estimate) %>% 
    mutate(
        pct_white = white / total_pop,
        pct_black = black / total_pop,
        pct_native = native / total_pop,
        pct_hispanic = hispanic / total_pop,
        pct_below_poverty = below_poverty / total_pop
    )

get popualtion density (for rough urban/rural calculation)

In [ ]:
%%R -o census_data

library(sf)

census_data = census_data %>%
  mutate(
    area_sqkm = st_area(geometry) / 10^6,  # area in square kilometers
    pop_density = total_pop / area_sqkm
  ) %>% 
  # remove geometry
  st_drop_geometry() 

Show dataframe by population density

In [ ]:
census_data.sort_values(by='pop_density', ascending=False)

## Join Election Data with Census Data

In [ ]:
df = election_results.merge(census_data, left_on='fips', right_on='GEOID', how='left')
df

The following plaes are excluded due to census API not returning data for them:

In [ ]:
# show all
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df.query('total_pop.isna()')[['fips','state','county']].drop_duplicates())

# drop these
df = df.query('total_pop.notna()')

⚠️ drop na values 

In [ ]:
all_columns_except_class = [x for x in df.columns if x != 'class']
# find rows with blanks
rows_with_na_values = df[df[all_columns_except_class].isna().any(axis=1)]
display(rows_with_na_values)

# drop those rows
df = df.dropna(subset=all_columns_except_class)


⚠️ drop connecticut, alaska and DC


In [ ]:
df = df.query('state != "Connecticut"')
df = df.query('state != "Alaska"')
df = df.query('state.notna()')

## Add region

In [ ]:
%%R  -i df
# Get state-level population
states_pop <- get_acs(
  geography = "state",
  variables = "B01003_001",
  year = 2022
)

# Add regions (manually or via a built-in lookup)
state_regions <- tibble::tibble(
  state = state.name,
  region = state.region
)


In [ ]:
%%R  -o df

# merge it onto census data
df <- df %>%
  left_join(state_regions, by = "state")

## Clean Up & Output to CSV

In [ ]:
df.drop(columns=['class', 'GEOID', 'NAME'], inplace=True)
df.to_csv('cleaned_and_merged_data.csv', index=False)
df